In [ ]:
# Import libraries
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt

from functools import reduce

In [ ]:
!pip install pip-tools firebase python_jwt gcloud sseclient pycrypto requests_toolbelt

from firebase import firebase
fb_app = firebase.FirebaseApplication('https://project-morphine-default-rtdb.asia-southeast1.firebasedatabase.app/', None)
firebase_db = fb_app.get('/', None)
# result = fb_app.get('/users', 1)
print(firebase_db)

{'ax': {'array0': [0.037668448, 0.037058927, 0.036815118, 0.038277969, 0.038643682, 0.036693214, 0.038521778, 0.037546544, 0.037790352, 0.038156065, 0.036083693, 0.037180831, 0.037668448, 0.038277969, 0.038156065, 0.03742464, 0.037058927, 0.037790352, 0.038643682, 0.037668448, 0.038277969, 0.037912257, 0.037546544, 0.037546544, 0.037912257, 0.036815118, 0.037912257, 0.038034161, 0.037912257, 0.036815118, 0.03742464, 0.036693214, 0.038521778, 0.038277969, 0.037790352, 0.037790352, 0.037912257, 0.037790352, 0.038277969, 0.037180831, 0.037668448, 0.037912257, 0.037668448, 0.037668448, 0.037302735, 0.037790352, 0.037912257, 0.038277969, 0.037546544, 0.037912257, 0.037546544, 0.037302735, 0.037668448, 0.037668448, 0.03742464, 0.036693214, 0.038034161, 0.037180831, 0.037302735, 0.038765586, 0.038034161, 0.03657131, 0.036815118, 0.038765586, 0.037058927, 0.03742464, 0.03742464, 0.03742464, 0.03742464, 0.037790352, 0.037668448, 0.037790352, 0.038156065, 0.038521778, 0.037180831, 0.037302735, 0

# Read data into dataframe

Firebase dictionary keys:

- Accelerometer: ax, ay, az
- Gyro: gx, gy, gz
- done: Set to 1 when all 20s of data is uploaded

subkeys: array0 to array3, each representing 5s of data

In [ ]:
# Accelero, Gyro, Done
mainKeys = list(firebase_db.keys())

# array0 to 3
subKeys = list(firebase_db[mainKeys[0]].keys())

print(mainKeys)
print(subKeys)

['ax', 'ay', 'az', 'done', 'fallPrediction', 'gx', 'gy', 'gz', 'pythonReceived', 'test']
['array0', 'array1', 'array2', 'array3']


In [ ]:
firebase_db['ax'].keys()

dict_keys(['array0', 'array1', 'array2', 'array3'])

In [ ]:
# x-axis acceleration
accel_x = pd.concat([pd.Series(v,name=k) for k,v in firebase_db['ax'].items()],axis=0)
# y-axis acceleration
accel_y = pd.concat([pd.Series(v,name=k) for k,v in firebase_db['ay'].items()],axis=0)
# z-axis acceleration
accel_z = pd.concat([pd.Series(v,name=k) for k,v in firebase_db['az'].items()],axis=0)

accel_db = pd.concat([accel_x,accel_y,accel_z],axis=1)
accel_db.columns = ['accel_x','accel_y','accel_z']

accel_db['accel_net'] = np.sqrt(accel_db['accel_x']**2 + accel_db['accel_y']**2 + accel_db['accel_z']**2)

accel_db

,accel_x,accel_y,accel_z,accel_net
0,0.037668,0.045714,0.972187,0.973990
1,0.037059,0.045592,0.972309,0.974083
2,0.036815,0.046202,0.972187,0.973980
3,0.038278,0.047055,0.974260,0.976146
4,0.038644,0.046689,0.973528,0.975413
...,...,...,...,...
95,0.037181,0.046324,0.973406,0.975217
96,0.037790,0.046324,0.973041,0.974875
97,0.037303,0.046567,0.972431,0.974260
98,0.037303,0.046446,0.972553,0.974376


In [ ]:
# TODO: Read Gyroscope data, if fall detection needs it someday

# Preprocessing to obtain summary statistics

In [ ]:
# Feature engineering
# Each SisFall data .txt file has a few thousand rows. This compresses each file into a single row, for ML
# (min/max/mean/var/skew/kurtosis) * ( (x/y/z) * (accel/rota) + net_accel) = 42 columns
def create_features(df):
#     from functools import reduce
    Min = df.min(axis=0).reset_index().rename({0:'min'},axis=1)
    Max = df.max(axis=0).reset_index().rename({0:'max'},axis=1)
    Mean = df.mean(axis=0).reset_index().rename({0:'mean'},axis=1)
    Var = df.var(axis=0).reset_index().rename({0:'variance'},axis=1)
    Skew = df.skew(axis=0).reset_index().rename({0:'skewness'},axis=1)
    Kurtosis = df.kurtosis(axis=0).reset_index().rename({0:'kurtosis'},axis=1)
    
    df_features = reduce(lambda left,right: pd.merge(left,right,on='index',)
                         ,[Min,Max,Mean,Var,Skew,Kurtosis])
    
    df_features.set_index('index',inplace=True)
    df_features = df_features.unstack().to_frame().sort_index(level=1).transpose()
    df_features.columns = df_features.columns.map('_'.join)

    return(df_features)

In [ ]:
create_features(accel_db)

,kurtosis_accel_net,max_accel_net,mean_accel_net,min_accel_net,skewness_accel_net,variance_accel_net,kurtosis_accel_x,max_accel_x,mean_accel_x,min_accel_x,...,mean_accel_y,min_accel_y,skewness_accel_y,variance_accel_y,kurtosis_accel_z,max_accel_z,mean_accel_z,min_accel_z,skewness_accel_z,variance_accel_z
0,3.990787,0.978972,0.974458,0.969119,-0.482058,0.000001,7.65627,0.042423,0.0376,0.034377,...,0.046983,0.044007,-0.210381,5.233848e-07,3.903533,0.977064,0.972597,0.967189,-0.489036,0.000001


# Fall Detection algorithm

In [ ]:
# Decision tree based on SisFall dataset. Can be further fine tuned

# Reads 1 line of summary statistics, to see if fall (1) or not fall(0)
def isFall(df):
    if df['skewness_accel_net'].item() <= 5.010115385055542:
        if df['max_accel_y'].item() <= 1.301025390625:
            return 0
        else:  # if max_y_accel_g > 1.301025390625
            if df['mean_accel_y'].item() <= -0.707072913646698:
                return 0
            else:  # if mean_y_accel_g > -0.707072913646698
                return 1
    else:  # if skewness_net_acceleration > 5.010115385055542
        if df['mean_accel_y'].item() <= -0.7913139462471008:
            return 0
        else:  # if mean_y_accel_g > -0.7913139462471008
            return 1

# Actual detection

Loops every 10s

In [ ]:
firebase_db.keys()

dict_keys(['ax', 'ay', 'az', 'done', 'fallPrediction', 'gx', 'gy', 'gz', 'pythonReceived', 'test'])

In [ ]:
import time

# Settings #
isRunning = True
checkFrequency = 9 # Checks firebase every 9s
fb_app = firebase.FirebaseApplication('https://project-morphine-default-rtdb.asia-southeast1.firebasedatabase.app/', None)
intervalCount = 0

# Saves the most recent block of 20s data, to maintain a rolling dataframe with 40s worth of data
previous_20s_data = pd.DataFrame()

# Script #
while isRunning:
    time_start = time.time()
    print('Checking Firebase for new data')
    
    done = fb_app.get('/done',None) # 1 if 20s of data ready, 0 otherwise
    if(done == 0): # Arduino not done sending 20s worth of data to Firebase
        time.sleep(checkFrequency)
        continue

    print('20s of data uploaded. Getting data')
    firebase_db = fb_app.get('/', None)
    # intervalCount += 1 # For errorchecking, can delete this
    
    print('Data received, creating dataframe')
    accel_x = pd.concat([pd.Series(v,name=k) for k,v in firebase_db['ax'].items()],axis=0)
    accel_y = pd.concat([pd.Series(v,name=k) for k,v in firebase_db['ay'].items()],axis=0)
    accel_z = pd.concat([pd.Series(v,name=k) for k,v in firebase_db['az'].items()],axis=0)

    # Update Firebase that Python has received the data
    toFirebase = {'pythonReceived':True}
    fb_app.patch('/',toFirebase)
    
    # Concat accel data together
    accel_db = pd.concat([accel_x,accel_y,accel_z],axis=1)
    accel_db.columns = ['accel_x','accel_y','accel_z']
    accel_db['accel_net'] = np.sqrt(accel_db['accel_x']**2 + accel_db['accel_y']**2 + accel_db['accel_z']**2)

    print('Predicting fall...')

    # Predicting fall based on recent 20s of data
    summary_statistics_20s = create_features(accel_db)
    fall_status_20s = bool( isFall(summary_statistics))

    # Predicting fall based on recent 40s of data
    previous_40s_data = pd.concat([accel_db, previous_data],axis=0)
    summary_statistics_40s = create_features(previous_40s_data)
    fall_status_40s = bool( isFall(summary_statistics))

    # Flags if fall was detected either in recent 20s of data or recent 40s of data
    print('!! Fall Prediction: ', fall_status_20s or fall_status_40s)
    
    # Sends prediction status to Firebase
    toFirebase = {'fallPrediction':fall_status}
    fb_app.patch('/',toFirebase)

    # Saves most recent 20s block of data, to be used in the next loop with the incoming 20s block of data
    previous_data = accel_db
    print("")


    # Make loop run every 10s regardless of computation time
    # If computation time >10s, sleep 0 seconds (i.e. don't sleep)
    time_end = time.time()
    time.sleep(max(0, checkFrequency - (time_end-time_start)))

Checking Firebase for new data
20s of data uploaded. Getting data
Data received, creating dataframe
Predicting fall...
!! Fall Prediction:  False

Checking Firebase for new data
20s of data uploaded. Getting data
Data received, creating dataframe
Predicting fall...
!! Fall Prediction:  False



KeyboardInterrupt: ignored

True

# TODO:

- Currently does not account for falls across 2 intervals of data. Need to store past 20s of data and run again, in an overlapping window